In [1]:
import os
import pandas as pd
from collections import defaultdict

In [2]:
# this is the location to the dataset
movie_ratings_filename = os.path.join("ml-100k", "u.data")

In [3]:
# this is the location to the dataset
movie_ratings_base = os.path.join("ml-100k", "ua.base")

In [4]:
movie_ratings = pd.read_csv(movie_ratings_base, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating", "Timestamp"])

In [5]:
movie_ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [6]:
# The date time above is not user readable. They are computed as seconds from epoch. 
# So we will convert it to readable format

In [7]:
# We now devise a condition that dictates favorable ratings

In [8]:
movie_ratings["Popular"] = movie_ratings["Rating"] > 3

In [9]:
movie_ratings.head()

,UserID,MovieID,Rating,Timestamp,Popular
0,1,1,5,874965758,True
1,1,2,3,876893171,False
2,1,3,4,878542960,True
3,1,4,3,876893119,False
4,1,5,3,889751712,False


In [10]:
ratings = movie_ratings.copy()

In [11]:
ratings.head()

,UserID,MovieID,Rating,Timestamp,Popular
0,1,1,5,874965758,True
1,1,2,3,876893171,False
2,1,3,4,878542960,True
3,1,4,3,876893119,False
4,1,5,3,889751712,False


In [12]:
favorable_ratings = ratings[ratings["Popular"]]

In [13]:
favorable_ratings.head()

,UserID,MovieID,Rating,Timestamp,Popular
0,1,1,5,874965758,True
2,1,3,4,878542960,True
5,1,6,5,887431973,True
6,1,7,4,875071561,True
8,1,9,5,878543541,True


In [14]:
# Favorable reviews by base set

In [15]:
favorable_reviews_by_users = dict((id, frozenset(vect.values)) for id, vect in favorable_ratings.groupby("UserID")["MovieID"])

In [16]:
num_favorable_by_movie = ratings[["MovieID", "Popular"]].groupby("MovieID").sum()

In [17]:
num_favorable_by_movie.sort_values(by="Popular", ascending=False).head()

,Popular
MovieID,
50,428.0
100,354.0
181,331.0
174,316.0
98,310.0


In [18]:
frequent_itemsets = {}

In [19]:
min_support = 150

In [20]:
#from collections import defaultdict
def find_frequent_itemsets(favorable_reviews_by_users, itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in itemsets:
            if itemset.issubset(reviews):
                for other_movie_reviewed in reviews - itemset:
                    new_superset = itemset | frozenset((other_movie_reviewed,))
                    counts[new_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

In [21]:
import sys
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 150

# Frequent Set will hold the frequent itemsets where index value indicates the size of the itemset

# Here we populate all Popular movies which havea favorable count which is greater than min_support required
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Popular"])
                                for movie_id, row in num_favorable_by_movie.iterrows()
                                if row["Popular"] > min_support)

print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))
sys.stdout.flush()


There are 69 movies with more than 150 favorable reviews


In [22]:
# we consider subset length of max 4
for k in range(2, 4):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}. Hence terminating...".format(k))
        sys.stdout.flush()
        break
    else:
        print("Found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
# Not interested in the itemsets of length 1
del frequent_itemsets[1]

Found 1193 frequent itemsets of length 2
Found 9906 frequent itemsets of length 3


In [23]:
# Now we create popular candidates for the association rules. 
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

There are 32104 candidate rules


In [24]:
# Here these movie ids make no sense so later on we will use u.items to fetch movie names
print(candidate_rules[:10])

[(frozenset({7}), 1), (frozenset({1}), 7), (frozenset({9}), 1), (frozenset({1}), 9), (frozenset({12}), 1), (frozenset({1}), 12), (frozenset({15}), 1), (frozenset({1}), 15), (frozenset({22}), 1), (frozenset({1}), 22)]


In [25]:
# Now, we compute the confidence of each of these rules. 
# these rules are formulated using movies which have favorable count atleast = >  min_support
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
# we maintain confidence of each rule in dictionary keyed by the rule
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}

In [26]:
# Choose only rules above a minimum confidence level
min_confidence = 0.9

In [27]:
# Filter out the rules with poor confidence
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))

67


In [28]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)

In [29]:
# We can now get the movie titles from the dataset
movie_name_filename = os.path.join("ml-100k", "u.item")
columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
movie_name_data = pd.read_csv(movie_name_filename, delimiter="|", header=None, encoding = "mac-roman", names=columns, usecols=["MovieID", "Title"])


In [30]:
movie_name_data.head()

,MovieID,Title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [31]:
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0] # this will remove type information that accompanies a value
    return title

In [32]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends Terminator, The (1984), Dead Poets Society (1989) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 0.962

Rule #2
Rule: If a person recommends Back to the Future (1985), 2001: A Space Odyssey (1968) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 0.960

Rule #3
Rule: If a person recommends When Harry Met Sally... (1989), Terminator, The (1984) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 0.954

Rule #4
Rule: If a person recommends Terminator, The (1984), Forrest Gump (1994) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 0.949

Rule #5
Rule: If a person recommends Sting, The (1973), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Confidence: 0.945



In [34]:
movie_ratings_test = os.path.join("ml-100k", "ua.test")
movie_ratings_test = pd.read_csv(movie_ratings_base, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating", "Timestamp"])
movie_ratings_test["Popular"] = movie_ratings_test["Rating"] > 3

In [35]:
# Evaluation using test data
# we use data of users other than the 200 we used for training
test_dataset = movie_ratings_test.copy()
test_favorable = test_dataset[test_dataset["Popular"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("UserID")["MovieID"])

In [36]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [37]:
test_confidence = {candidate_rule:
                   (correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule]))
                   for candidate_rule in rule_confidence}
print(len(test_confidence))

67


In [38]:
for index in range(10):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")

Rule #1
Rule: If a person recommends Terminator, The (1984), Dead Poets Society (1989) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 0.962
 - Test Confidence: 0.962

Rule #2
Rule: If a person recommends Back to the Future (1985), 2001: A Space Odyssey (1968) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 0.960
 - Test Confidence: 0.960

Rule #3
Rule: If a person recommends When Harry Met Sally... (1989), Terminator, The (1984) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 0.954
 - Test Confidence: 0.954

Rule #4
Rule: If a person recommends Terminator, The (1984), Forrest Gump (1994) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 0.949
 - Test Confidence: 0.949

Rule #5
Rule: If a person recommends Sting, The (1973), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Train Confidence: 0.945
 - Test Confidence: 0.945

Rule #6
Rule: If a person rec